# AWS Retail Sales Forecasting - Model Training Notebook

This notebook trains Prophet and XGBoost models for sales forecasting using data from AWS Athena.

In [ ]:
# Cell 1: Install dependencies
!pip install prophet pyathena scikit-learn

In [ ]:
# Cell 2: Import libraries
import pandas as pd
import numpy as np
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import boto3
from pyathena import connect
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Cell 3: Connect to Athena and read data
bucket = 'forcasting1'
region = 'us-east-2'
conn = connect(
    s3_staging_dir=f's3://{bucket}/athena-results/',
    region_name=region,
    schema_name='sales_forecast_db'
)
query = """
SELECT
    date,
    product_id,
    store_id,
    sales_quantity,
    sales_lag1,
    sales_lag7,
    sales_rolling_7d,
    promotion,
    is_weekend
FROM processed_data
WHERE sales_lag7 IS NOT NULL
ORDER BY product_id, store_id, date
"""
df = pd.read_sql(query, conn)
df['date'] = pd.to_datetime(df['date'])
print(f"✅ Loaded {len(df)} records")

In [ ]:
# Cell 4: Split train/test sets (last 14 days as test)
train_df = df[df['date'] < df['date'].max() - pd.Timedelta(days=14)]
test_df = df[df['date'] >= df['date'].max() - pd.Timedelta(days=14)]
print(f"Training set: {len(train_df)} records ({train_df['date'].min()} to {train_df['date'].max()})")
print(f"Test set: {len(test_df)} records ({test_df['date'].min()} to {test_df['date'].max()})")

In [ ]:
# Cell 5: Baseline model (moving average)
def baseline_forecast(train_data, test_data, window=7):
    predictions = []
    for _, row in test_data.iterrows():
        historical = train_data[
            (train_data['product_id'] == row['product_id']) &
            (train_data['store_id'] == row['store_id'])
        ].tail(window)
        pred = historical['sales_quantity'].mean()
        predictions.append(pred)
    return np.array(predictions)
baseline_pred = baseline_forecast(train_df, test_df, window=7)
baseline_mape = mean_absolute_percentage_error(test_df['sales_quantity'], baseline_pred) * 100
print(f"📊 Baseline MAPE: {baseline_mape:.2f}%")

In [ ]:
# Cell 6: Train Prophet model
def train_prophet_model(product, store, data):
    subset = data[
        (data['product_id'] == product) &
        (data['store_id'] == store)
    ][['date', 'sales_quantity', 'promotion', 'is_weekend']].copy()
   
    subset.rename(columns={'date': 'ds', 'sales_quantity': 'y'}, inplace=True)
   
    model = Prophet(
        yearly_seasonality=False,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode='additive'
    )
    model.add_regressor('promotion')
    model.add_regressor('is_weekend')
   
    model.fit(subset)
    return model

In [ ]:
# Cell 7: Train all product-store combinations
models = {}
products = train_df['product_id'].unique()
stores = train_df['store_id'].unique()
for product in products:
    for store in stores:
        key = f"{product}_{store}"
        print(f"Training {key}...")
        models[key] = train_prophet_model(product, store, train_df)
print(f"✅ Training completed for {len(models)} models")

In [ ]:
# Cell 8: Generate test set predictions
prophet_predictions = []
for _, row in test_df.iterrows():
    key = f"{row['product_id']}_{row['store_id']}"
    model = models[key]
   
    future = pd.DataFrame({
        'ds': [row['date']],
        'promotion': [row['promotion']],
        'is_weekend': [row['is_weekend']]
    })
   
    forecast = model.predict(future)
    prophet_predictions.append(forecast['yhat'].iloc[0])
prophet_pred = np.array(prophet_predictions)
prophet_mape = mean_absolute_percentage_error(test_df['sales_quantity'], prophet_pred) * 100
print(f"📊 Prophet MAPE: {prophet_mape:.2f}%")

In [ ]:
# Cell 9: Model comparison
improvement = ((baseline_mape - prophet_mape) / baseline_mape) * 100
print(f"\n🎯 Model performance comparison:")
print(f"Baseline MAPE: {baseline_mape:.2f}%")
print(f"Prophet MAPE: {prophet_mape:.2f}%")
print(f"Accuracy improvement: {improvement:.1f}%")

In [ ]:
# Cell 10: Visualization (select one product-store)
test_sample = test_df[
    (test_df['product_id'] == 'A') &
    (test_df['store_id'] == 'Store1')
].copy()
start_idx = test_sample.index[0]
end_idx = test_sample.index[-1] + 1
test_sample['prophet_pred'] = prophet_pred[start_idx:end_idx]
test_sample['baseline_pred'] = baseline_pred[start_idx:end_idx]
plt.figure(figsize=(12, 6))
plt.plot(test_sample['date'], test_sample['sales_quantity'], 'o-', label='Actual', linewidth=2)
plt.plot(test_sample['date'], test_sample['prophet_pred'], 's--', label='Prophet', linewidth=2)
plt.plot(test_sample['date'], test_sample['baseline_pred'], '^--', label='Baseline', linewidth=2)
plt.legend()
plt.title('Product A - Store1: Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('Sales Quantity')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('forecast_comparison.png', dpi=150)
plt.show()

In [ ]:
# Cell 11: Save models to S3
import joblib
for key, model in models.items():
    model_path = f'/tmp/{key}_model.joblib'
    joblib.dump(model, model_path)
   
    s3_path = f's3://{bucket}/models/{key}_model.joblib'
    boto3.client('s3').upload_file(model_path, bucket, f'models/{key}_model.joblib')
    print(f"✅ Saved {key}")
print("✅ All models uploaded to S3")

## XGBoost Model Training

In [ ]:
!pip install xgboost pyathena scikit-learn --quiet

In [ ]:
# 2. Import libraries
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_percentage_error
from pyathena import connect
import matplotlib.pyplot as plt
import boto3
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 3. Connect to Athena and read processed data (specify schema)
bucket = 'forcasting1'          # Change to your bucket
region = 'us-east-2'

conn = connect(
    s3_staging_dir=f's3://{bucket}/athena-results/',
    region_name=region,
    schema_name='sales_forecast_db'   # Key: specify database
)

query = """
SELECT
    date,
    product_id,
    store_id,
    sales_quantity,
    sales_lag1,
    sales_lag7,
    sales_rolling_7d,
    sales_rolling_30d,
    promotion,
    is_weekend,
    day_of_week
FROM processed_data                  # Change if table name differs
WHERE sales_lag7 IS NOT NULL
ORDER BY product_id, store_id, date
"""

df = pd.read_sql(query, conn)
df['date'] = pd.to_datetime(df['date'])
print(f"✅ Loaded {len(df)} records")
display(df.head())

In [ ]:
# 4. Feature engineering + encoding
le_product = LabelEncoder()
le_store = LabelEncoder()

df['product_id_enc'] = le_product.fit_transform(df['product_id'])
df['store_id_enc'] = le_store.fit_transform(df['store_id'])

# Ensure day_of_week exists (extract from date if not)
if 'day_of_week' not in df.columns:
    df['day_of_week'] = df['date'].dt.weekday

features = [
    'product_id_enc', 'store_id_enc',
    'sales_lag1', 'sales_lag7',
    'sales_rolling_7d', 'sales_rolling_30d',
    'promotion', 'is_weekend',
    'day_of_week'
]

target = 'sales_quantity'

In [ ]:
# 5. Split train/test sets (last 14 days as test, consistent with Prophet)
train_df = df[df['date'] < df['date'].max() - pd.Timedelta(days=14)]
test_df = df[df['date'] >= df['date'].max() - pd.Timedelta(days=14)]

X_train = train_df[features]
y_train = train_df[target]
X_test = test_df[features]
y_test = test_df[target]

print(f"Training set: {X_train.shape[0]} records")
print(f"Test set: {X_test.shape[0]} records")

In [ ]:
# 6. Train XGBoost (single model, overall effect)
xgb_model = xgb.XGBRegressor(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    objective='reg:squarederror',
    n_jobs=-1
)

xgb_model.fit(X_train, y_train)

In [ ]:
# 7. Predict + MAPE
xgb_pred = xgb_model.predict(X_test)
xgb_mape = mean_absolute_percentage_error(y_test, xgb_pred) * 100

print(f"\n📊 XGBoost MAPE: {xgb_mape:.2f}%")
print(f"   Prophet MAPE: 12.94%")
print(f"   Baseline MAPE: 22.70%")
if xgb_mape < 12.94:
    print(f"🎉 Improvement over Prophet: {(12.94 - xgb_mape)/12.94*100:.1f}%!")
else:
    print(f"   Prophet slightly better")

In [ ]:
# 8. Feature importance visualization
plt.figure(figsize=(10, 6))
xgb.plot_importance(xgb_model, max_num_features=10, importance_type='gain')
plt.title('XGBoost Feature Importance (Gain)')
plt.show()

In [ ]:
# 9. Visualize forecast vs actual (Product A - Store1)
test_sample = test_df[
    (test_df['product_id'] == 'A') &
    (test_df['store_id'] == 'Store1')
].copy().reset_index(drop=True)

# Extract corresponding predictions
start_idx = test_sample.index[0]
end_idx = test_sample.index[-1] + 1
sample_pred = xgb_pred[start_idx:end_idx]

plt.figure(figsize=(12, 6))
plt.plot(test_sample['date'], test_sample['sales_quantity'], 'o-', label='Actual', linewidth=2)
plt.plot(test_sample['date'], sample_pred, 's--', label='XGBoost', color='green', linewidth=2)
plt.plot(test_sample['date'], test_sample['sales_rolling_7d'], '--', label='Rolling 7d Avg', alpha=0.7, color='gray')
plt.legend()
plt.title('Product A - Store1: XGBoost Forecast vs Actual')
plt.xlabel('Date')
plt.ylabel('Sales Quantity')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Generate Future 30-Day Predictions

In [ ]:
# 1. Generate future date sequence
future_dates = pd.date_range(start='2024-07-01', end='2024-07-30', freq='D')
print(f"Predicting future {len(future_dates)} days: {future_dates[0]} to {future_dates[-1]}")

In [ ]:
# 2. Generate future data for each product-store combination
future_list = []
for product in df['product_id'].unique():
    for store in df['store_id'].unique():
        # Use last historical record as base
        last_record = df[
            (df['product_id'] == product) &
            (df['store_id'] == store)
        ].sort_values('date').iloc[-1]
        
        for future_date in future_dates:
            # Construct features (key: fill lag/rolling with historical values)
            new_row = {
                'date': future_date,
                'product_id': product,
                'store_id': store,
                'product_id_enc': le_product.transform([product])[0],
                'store_id_enc': le_store.transform([store])[0],
                'day_of_week': future_date.weekday(),
                # Lag and rolling need historical values known at prediction time, simplified using last day's value
                'sales_lag1': last_record['sales_quantity'],
                'sales_lag7': last_record['sales_quantity'],
                'sales_rolling_7d': last_record['sales_rolling_7d'] if not pd.isna(last_record['sales_rolling_7d']) else last_record['sales_quantity'],
                'sales_rolling_30d': last_record['sales_rolling_30d'] if not pd.isna(last_record['sales_rolling_30d']) else last_record['sales_quantity'],
                # Assume no promotion in future (conservative), or customize rules
                'promotion': 0,
                'is_weekend': 1 if future_date.weekday() >= 5 else 0
            }
            future_list.append(new_row)

future_df = pd.DataFrame(future_list)
print(f"Generated future data {len(future_df)} records (15 combinations × 30 days)")

In [ ]:
# 3. Predict with XGBoost
future_X = future_df[features]
future_pred = xgb_model.predict(future_X)
future_df['predicted_sales'] = future_pred.round(2)

In [ ]:
# 4. Visualize total sales forecast trend
daily_total = future_df.groupby('date')['predicted_sales'].sum().reset_index()

plt.figure(figsize=(14, 6))
plt.plot(daily_total['date'], daily_total['predicted_sales'], 'o-', linewidth=2, markersize=4)
plt.title('July 2024 Total Store Sales Forecast (XGBoost)', fontsize=16)
plt.xlabel('Date')
plt.ylabel('Predicted Total Sales')
plt.grid(True, alpha=0.3)
plt.xticks(rotation=45)
# Mark weekends
weekends = daily_total[daily_total['date'].dt.weekday >= 5]
plt.scatter(weekends['date'], weekends['predicted_sales'], color='red', s=50, label='Weekend', zorder=5)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# 5. Output summary statistics
print("\n📊 July Forecast Total Sales Statistics:")
print(f"Total predicted sales: {future_df['predicted_sales'].sum():.0f}")
print(f"Daily average sales: {future_df['predicted_sales'].mean():.1f}")
print(f"Highest single day: {daily_total['predicted_sales'].max():.0f} ({daily_total.loc[daily_total['predicted_sales'].idxmax(), 'date'].date()})")
print(f"Lowest single day: {daily_total['predicted_sales'].min():.0f} ({daily_total.loc[daily_total['predicted_sales'].idxmin(), 'date'].date()})")

In [ ]:
# 6. Summarize by product (which grows fastest)
product_total = future_df.groupby('product_id')['predicted_sales'].sum().sort_values(ascending=False)
print("\n🔥 July Predicted Total Sales by Product Ranking:")
for product, sales in product_total.items():
    print(f"  {product}: {sales:.0f}")

In [ ]:
# 7. Save forecast results to S3 (optional)
future_df.to_csv('future_30days_forecast.csv', index=False)
boto3.client('s3').upload_file('future_30days_forecast.csv', bucket, 'predictions/future_30days_forecast.csv')
print(f"\n✅ Forecast results saved to s3://{bucket}/predictions/future_30days_forecast.csv")

# AWS Retail Sales Forecasting - Model Training Notebook

This notebook demonstrates training machine learning models for sales forecasting using XGBoost and Prophet.

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from prophet import Prophet
import matplotlib.pyplot as plt
import boto3
import joblib

## Load and Preprocess Data

In [ ]:
# Load data (assuming local CSV for demo)
df = pd.read_csv('../sales_data.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values(['product_id', 'store_id', 'date'])

# Basic preprocessing
df = df.dropna()
print(df.head())

## Feature Engineering

In [ ]:
# Create lag features
for lag in [1, 7, 14]:
    df[f'sales_lag{lag}'] = df.groupby(['product_id', 'store_id'])['sales_quantity'].shift(lag)

# Rolling statistics
df['sales_rolling_7d'] = df.groupby(['product_id', 'store_id'])['sales_quantity'].rolling(7).mean().reset_index(0, drop=True)
df['sales_rolling_30d'] = df.groupby(['product_id', 'store_id'])['sales_quantity'].rolling(30).mean().reset_index(0, drop=True)

# One-hot encode categorical
df = pd.get_dummies(df, columns=['product_id', 'store_id'])

# Drop rows with NaN from lags
df = df.dropna()
print(df.columns)

## Train XGBoost Model

In [ ]:
# Prepare features and target
features = [col for col in df.columns if col not in ['date', 'sales_quantity']]
X = df[features]
y = df['sales_quantity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost
model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, max_depth=5)
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print(f'XGBoost MAE: {mae:.2f}, RMSE: {rmse:.2f}')

## Train Prophet Model

In [ ]:
# Prepare data for Prophet (example for one product-store)
sample_df = df[(df['product_id_A'] == 1) & (df['store_id_Store1'] == 1)][['date', 'sales_quantity']]
sample_df.columns = ['ds', 'y']

# Train Prophet
prophet_model = Prophet(weekly_seasonality=True)
prophet_model.fit(sample_df)

# Forecast
future = prophet_model.make_future_dataframe(periods=30)
forecast = prophet_model.predict(future)
print(forecast[['ds', 'yhat']].tail())

## Model Evaluation and Comparison

In [ ]:
# Compare models (simplified)
# For XGBoost
print(f'XGBoost Performance: MAE={mae:.2f}, RMSE={rmse:.2f}')

# For Prophet (on sample)
actual = sample_df['y'].tail(30)
predicted = forecast['yhat'].tail(30)
prophet_mae = mean_absolute_error(actual, predicted)
print(f'Prophet Performance: MAE={prophet_mae:.2f}')

# Plot
plt.figure(figsize=(10, 6))
plt.plot(sample_df['ds'], sample_df['y'], label='Actual')
plt.plot(forecast['ds'], forecast['yhat'], label='Predicted')
plt.legend()
plt.show()

## Save Models

In [ ]:
# Save XGBoost model
joblib.dump(model, '../models/xgboost_model.joblib')

# Save Prophet model
prophet_model.save('../models/prophet_model.json')

print("Models saved successfully!")